In [25]:
import pandas as pd

myeloid_sample_info = pd.read_csv("~/R/Stemformatics/activation_atlas/stemformatics_atlas_activation.0.1.samples.tsv", sep='\t', skipinitialspace=True)
myeloid_sample_info.rename(columns={myeloid_sample_info.columns[0]: 'sample_id'}, inplace=True)
myeloid_sample_info.set_index('sample_id',  inplace=True)

myeloid_sample_info

,cell_type,parental_cell_type,cell_line,treatment_broad,receptor_broad,receptor,dataset_id,build_or_project,library_kit_broad,library_kit,platform,capture,library_layout
sample_id,,,,,,,,,,,,,
2865_GSM1565955,dendritic cell,monocyte,NaN,bacterial,WP myco,WP myco,2865,build,Illumina,TruSeq RNA Sample Preparation Kit v2,Illumina HiSeq 2500,poly A,50bp single
2865_GSM1565956,dendritic cell,monocyte,NaN,control,control,control,2865,build,Illumina,TruSeq RNA Sample Preparation Kit v2,Illumina HiSeq 2500,poly A,50bp single
2865_GSM1565957,dendritic cell,monocyte,NaN,bacterial,WP myco,WP myco,2865,build,Illumina,TruSeq RNA Sample Preparation Kit v2,Illumina HiSeq 2500,poly A,50bp single
2865_GSM1565958,dendritic cell,monocyte,NaN,control,control,control,2865,build,Illumina,TruSeq RNA Sample Preparation Kit v2,Illumina HiSeq 2500,poly A,50bp single
2865_GSM1565959,dendritic cell,monocyte,NaN,bacterial,WP myco,WP myco,2865,build,Illumina,TruSeq RNA Sample Preparation Kit v2,Illumina HiSeq 2500,poly A,50bp single
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786_GSM4819707,macrophage,monocyte,THP-1,primed,primed,EP2 + IFNGR + TLR4,9786,build,Illumina,TruSeq RNA Sample Preparation Kit,Illumina HiSeq X Ten,poly A,150bp paired
9786_GSM4819708,macrophage,monocyte,THP-1,primed,primed,EP2 + IFNGR + TLR4,9786,build,Illumina,TruSeq RNA Sample Preparation Kit,Illumina HiSeq X Ten,poly A,150bp paired
9786_GSM4819709,macrophage,monocyte,THP-1,intrinsic,intrinsic + cytokine_chemokine,EP2 + IL4R + IL13R,9786,build,Illumina,TruSeq RNA Sample Preparation Kit,Illumina HiSeq X Ten,poly A,150bp paired


In [26]:
dataset_id_list = myeloid_sample_info['dataset_id'].unique()

import glob
import os

integrated_activation_atlas = {}

for df_id in dataset_id_list:
    f = glob.glob(os.path.join('/home/chzhan1/R/Stemformatics/expression_files', str(df_id),'*.raw.tsv'))
    if len(f)>0:
        temp_df = pd.read_csv(f[0], sep='\t')
        if temp_df.columns[0] == "Unnamed: 0":
            temp_df.rename(columns={temp_df.columns[0]: 'gene_names'}, inplace=True)
            temp_df.set_index('gene_names', inplace=True)        
        integrated_activation_atlas[df_id] = temp_df


In [27]:
integrated_activation_atlas.keys()

dict_keys([2865, 6075, 6412, 6530, 6729, 7079, 7193, 7264, 7393, 9756, 9758, 9759, 9760, 9762, 9768, 9771, 9775, 9776, 9786])

In [28]:
import torch
device = 'gpu' if torch.cuda.is_available() else 'cpu'
device

'gpu'

In [29]:
torch.cuda.device_count()

1

In [30]:
bio_tag = 'cell_type'
batch_tag = 'prep_method_broad'

In [31]:
import scvi

# Load the pre-trained model from the saved path
pt_model = scvi.model.SCVI.load("/home/chzhan1/Python/saved_best_model")

INFO     File /home/chzhan1/Python/saved_best_model/model.pt already downloaded                                    


/home/chzhan1/.local/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/chzhan1/.local/lib/python3.9/site-packages/ipy ...

/home/chzhan1/.local/lib/python3.9/site-packages/scvi/model/base/_utils.py:66: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via 

In [32]:
pt_model

SCVI model with the following parameters: 
n_hidden: 256, n_latent: 3, n_layers: 3, dropout_rate: 0.1, dispersion: gene-label, gene_likelihood: nb, 
latent_distribution: norm.
Training status: Trained
Model's adata is minified?: False

In [33]:
query_df = integrated_activation_atlas[2865].T
query_df

gene_names,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,ENSG00000268020,ENSG00000240361,ENSG00000186092,ENSG00000238009,...,ENSG00000273532,ENSG00000276351,ENSG00000275661,ENSG00000277856,ENSG00000275063,ENSG00000271254,ENSG00000275405,ENSG00000275987,ENSG00000277475,ENSG00000268674
2865_GSM1565955,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,9,0,0,0,0
2865_GSM1565956,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0
2865_GSM1565957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,18,0,0,0,0
2865_GSM1565958,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,2,37,0,0,0,0
2865_GSM1565959,0,4,0,0,0,0,0,0,0,3,...,0,0,0,0,0,14,0,0,0,0
2865_GSM1565960,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,23,0,0,0,0
2865_GSM1565961,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,4,42,0,0,0,0
2865_GSM1565962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,129,0,0,1,0
2865_GSM1565963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,45,0,0,0,0
2865_GSM1565964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,88,0,0,0,0


In [34]:
import scanpy as sc
query_adata = sc.AnnData(query_df)

In [35]:
query_adata.var_names

Index(['ENSG00000223972', 'ENSG00000227232', 'ENSG00000278267',
       'ENSG00000243485', 'ENSG00000284332', 'ENSG00000237613',
       'ENSG00000268020', 'ENSG00000240361', 'ENSG00000186092',
       'ENSG00000238009',
       ...
       'ENSG00000273532', 'ENSG00000276351', 'ENSG00000275661',
       'ENSG00000277856', 'ENSG00000275063', 'ENSG00000271254',
       'ENSG00000275405', 'ENSG00000275987', 'ENSG00000277475',
       'ENSG00000268674'],
      dtype='object', name='gene_names', length=58302)

In [36]:
scvi.model.SCVI.prepare_query_anndata(query_adata, pt_model)

INFO     Found 100.0% reference vars in query data.                                                                


In [37]:
query_adata.obs[batch_tag] = 'query'
query_adata.obs[bio_tag] = 'query'

model_q = scvi.model.SCVI.load_query_data(
    query_adata, pt_model
)

/home/chzhan1/.local/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/chzhan1/.local/lib/python3.9/site-packages/ipy ...



In [38]:
model_q.train(max_epochs=300, plan_kwargs=dict(weight_decay=0.0), early_stopping=True)

/home/chzhan1/.local/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/chzhan1/.local/lib/python3.9/site-packages/ipy ...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/chzhan1/.local/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/chzhan1/.local/lib/python3.9/site-packages/ipy ...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/chzhan1/.local/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/

Epoch 187/300:  62%|██████▏   | 187/300 [00:01<00:01, 95.61it/s, v_num=1, train_loss_step=3.9e+5, train_loss_epoch=3.9e+5]  
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 142646.906. Signaling Trainer to stop.


In [39]:
query_matrix = model_q.get_latent_representation()

query_matrix = pd.DataFrame(query_matrix)

query_matrix['sample_id'] = query_df.index
query_matrix.set_index('sample_id', inplace=True)
query_matrix.columns = query_matrix.columns = ['V' + str(col) for col in query_matrix.columns]

query_matrix

,V0,V1,V2
sample_id,,,
2865_GSM1565955,0.423826,-3.842799,-0.269434
2865_GSM1565956,-0.375050,-4.684259,-2.535945
2865_GSM1565957,0.312011,-3.069587,0.314131
2865_GSM1565958,-0.255916,-3.440376,-1.674236
2865_GSM1565959,0.051350,-3.279307,-0.694446
2865_GSM1565960,-0.230015,-3.202575,-1.102441
2865_GSM1565961,0.579176,-3.128626,0.345886
2865_GSM1565962,-0.051856,-2.862960,-0.889478
2865_GSM1565963,0.449840,-3.749729,-0.086407


In [40]:
scvi_matrix = pt_model.get_latent_representation()
scvi_matrix = pd.DataFrame(scvi_matrix)
scvi_matrix['sample_id'] = pt_model.adata.obs.index
scvi_matrix.set_index('sample_id', inplace=True)
scvi_matrix.columns = scvi_matrix.columns = ['V' + str(col) for col in scvi_matrix.columns]
scvi_matrix

,V0,V1,V2
sample_id,,,
1611_GSM2027310,4.033545,-1.157732,4.244139
1611_GSM2027311,4.415716,-0.556770,5.411424
1611_GSM2027312,1.538134,0.870058,3.068065
1611_GSM2027313,2.529716,0.425173,3.480353
1611_GSM2027314,3.978914,-1.058332,4.175323
...,...,...,...
9798_Kolf2_iMacs_UT_16h_2_01,0.210513,0.147489,3.473737
9798_Kolf2_iMacs_UT_16h_3_01,0.195405,0.184106,3.382013
9798_Kolf2_iMacs_UT_6h_1_01,0.258679,0.299569,3.524895


In [41]:
# Merge DataFrames by 'sample_id'
scvi_representation_coordinates_merged = pd.merge(scvi_matrix, pt_model.adata.obs, left_index=True, right_index=True)

scvi_representation_coordinates_merged

,V0,V1,V2,dataset_id,cell_type,parental_cell_type,final_cell_type,disease_state,organism,sample_type,...,capture,fragmentation,stranded,sequencing_layout,average_read_length,sequencer,training_atlas,sample_id_without_datasetid,_scvi_batch,_scvi_labels
1611_GSM2027310,4.033545,-1.157732,4.244139,1611,conventional dendritic cell,NaN,NaN,normal,homo sapiens,cDC2,...,oligo-dT beads,RNA heat and buffer,non stranded,paired,51,Illumina HiSeq 2000,include,GSM2027310,2,0
1611_GSM2027311,4.415716,-0.556770,5.411424,1611,conventional dendritic cell,NaN,NaN,normal,homo sapiens,cDC1,...,oligo-dT beads,RNA heat and buffer,non stranded,paired,51,Illumina HiSeq 2000,include,GSM2027311,2,0
1611_GSM2027312,1.538134,0.870058,3.068065,1611,plasmacytoid dendritic cell,NaN,NaN,normal,homo sapiens,pDC,...,oligo-dT beads,RNA heat and buffer,non stranded,paired,51,Illumina HiSeq 2000,include,GSM2027312,2,10
1611_GSM2027313,2.529716,0.425173,3.480353,1611,plasmacytoid dendritic cell,NaN,NaN,normal,homo sapiens,pDC,...,oligo-dT beads,RNA heat and buffer,non stranded,paired,50,Illumina HiSeq 2000,include,GSM2027313,2,10
1611_GSM2027314,3.978914,-1.058332,4.175323,1611,conventional dendritic cell,NaN,NaN,normal,homo sapiens,cDC2,...,oligo-dT beads,RNA heat and buffer,non stranded,paired,51,Illumina HiSeq 2000,include,GSM2027314,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9798_Kolf2_iMacs_UT_16h_2_01,0.210513,0.147489,3.473737,9798,macrophage,induced pluripotent stem cell,macrophage,normal,homo sapiens,UT_16h,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0,6
9798_Kolf2_iMacs_UT_16h_3_01,0.195405,0.184106,3.382013,9798,macrophage,induced pluripotent stem cell,macrophage,normal,homo sapiens,UT_16h,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0,6
9798_Kolf2_iMacs_UT_6h_1_01,0.258679,0.299569,3.524895,9798,macrophage,induced pluripotent stem cell,macrophage,normal,homo sapiens,UT_6h,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0,6
9798_Kolf2_iMacs_UT_6h_2_01,0.110319,0.145117,3.484755,9798,macrophage,induced pluripotent stem cell,macrophage,normal,homo sapiens,UT_6h,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0,6


In [42]:
query_matrix[bio_tag] = 'query'
query_matrix['parental_cell_type'] = 'query'
query_matrix['sample_source'] = 'query'

query_matrix['ref_or_query'] = 'query'
scvi_representation_coordinates_merged['ref_or_query'] = 'ref'

In [43]:
result = pd.concat([query_matrix, scvi_representation_coordinates_merged], axis=0, sort=False)
result

,V0,V1,V2,cell_type,parental_cell_type,sample_source,ref_or_query,dataset_id,final_cell_type,disease_state,...,capture,fragmentation,stranded,sequencing_layout,average_read_length,sequencer,training_atlas,sample_id_without_datasetid,_scvi_batch,_scvi_labels
2865_GSM1565955,0.423826,-3.842799,-0.269434,query,query,query,query,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865_GSM1565956,-0.375050,-4.684259,-2.535945,query,query,query,query,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865_GSM1565957,0.312011,-3.069587,0.314131,query,query,query,query,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865_GSM1565958,-0.255916,-3.440376,-1.674236,query,query,query,query,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865_GSM1565959,0.051350,-3.279307,-0.694446,query,query,query,query,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9798_Kolf2_iMacs_UT_16h_2_01,0.210513,0.147489,3.473737,macrophage,induced pluripotent stem cell,in vitro,ref,9798.0,macrophage,normal,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0.0,6.0
9798_Kolf2_iMacs_UT_16h_3_01,0.195405,0.184106,3.382013,macrophage,induced pluripotent stem cell,in vitro,ref,9798.0,macrophage,normal,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0.0,6.0
9798_Kolf2_iMacs_UT_6h_1_01,0.258679,0.299569,3.524895,macrophage,induced pluripotent stem cell,in vitro,ref,9798.0,macrophage,normal,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0.0,6.0
9798_Kolf2_iMacs_UT_6h_2_01,0.110319,0.145117,3.484755,macrophage,induced pluripotent stem cell,in vitro,ref,9798.0,macrophage,normal,...,unknown,unknown,unknown,unknown,unknown,unknown,include,Kolf2,0.0,6.0


In [44]:
import plotly.express as px
import plotly.io as pio

# Set default renderer to 'notebook'
pio.renderers.default = 'vscode'

fig = px.scatter_3d(
    result,
    x='V0',        # Column for x-axis
    y='V1',        # Column for y-axis
    z='V2',        # Column for z-axis
    color= bio_tag,  # Color points based on bio_tag
    symbol='ref_or_query',  # Set symbols based on ref_or_query
    symbol_map = dict(ref = 'circle', query = 'diamond-open'),
    color_discrete_map = dict(query = 'black')
)

# Update the layout to distinguish the categories better
fig.update_traces(marker=dict(size=4))  # Adjust the size as needed

# Update layout to customize the plot
fig.update_layout(
    height=500,  # Height of the plot in pixels
    width=800,   # Width of the plot in pixels
    margin=dict(t=20, l=0, r=0, b=0),  # Custom margins
    uirevision= True,
    hovermode = 'closest',
    scene_camera=dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.25, y=1.25, z=1.25)
    ),
    legend_title=dict(text='Cell Type')
)

for trace in fig.data:
  trace.name = trace.name.split(',')[0]

# Show the plot
fig.show()

In [45]:
fig = px.scatter_3d(
    result,
    x='V0',        # Column for x-axis
    y='V1',        # Column for y-axis
    z='V2',        # Column for z-axis
    color= 'parental_cell_type',  # Color points based on bio_tag
    symbol='ref_or_query',  # Set symbols based on ref_or_query
    symbol_map = dict(ref = 'circle', query = 'diamond-open'),
    color_discrete_map = dict(query = 'black')
)

# Update the layout to distinguish the categories better
fig.update_traces(marker=dict(size=4))  # Adjust the size as needed

# Update layout to customize the plot
fig.update_layout(
    height=500,  # Height of the plot in pixels
    width=800,   # Width of the plot in pixels
    margin=dict(t=20, l=0, r=0, b=0),  # Custom margins
    uirevision= True,
    hovermode = 'closest',
    scene_camera=dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.25, y=1.25, z=1.25)
    ),
    legend_title=dict(text='Parental Cell Type')
)

for trace in fig.data:
  trace.name = trace.name.split(',')[0]

# Show the plot
fig.show()